In [24]:
# First two lines should be commented out in very first run, uncomment after to
# reload after making changes in github
%cd ..
!rm -rf /content/RL-Chess-Transformers/
!git clone https://github.com/EmilGou/RL-Chess-Transformers.git
!rm -rf /content/RL-Chess-Transformers/GRPO/colab_notebooks

%cd /content/RL-Chess-Transformers/

/content
Cloning into 'RL-Chess-Transformers'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 197 (delta 120), reused 135 (delta 59), pack-reused 0 (from 0)
Receiving objects: 100% (197/197), 3.11 MiB | 12.64 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/RL-Chess-Transformers


In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install python-chess cairosvg
import torch
from torch import nn
import torch.nn.functional as F
import chess

In [4]:
# download raw moves file
!pip install -U -q gdown
FILE_ID="1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO"
!gdown --id $FILE_ID -O moves

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO
From (redirected): https://drive.google.com/uc?id=1BSBuF2dKOnVWuR5CNjp-o7QBYb-10JTO&confirm=t&uuid=3f8ac8e0-4336-4536-aef6-d2840b01ffbd
To: /content/RL-Chess-Transformers/moves
100% 134M/134M [00:02<00:00, 54.6MB/s]


In [5]:
moves = open("moves", "r").read()
moves = moves.split('\n\n')[:-1]
GAMES = [m.split('\n')[:-1] for m in moves]

In [6]:
from GRPO.model import AutoregressiveTransformer, ChessConfig

model = AutoregressiveTransformer(ChessConfig())
model


AutoregressiveTransformer(
  (token_emb): Embedding(2008, 1024, padding_idx=2006)
  (pos_emb): Embedding(257, 1024)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=4096, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=1024, out_features=2008, bias=False)
)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

AutoregressiveTransformer(
  (token_emb): Embedding(2008, 1024, padding_idx=2006)
  (pos_emb): Embedding(257, 1024)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=4096, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=1024, out_features=2008, bias=False)
)

In [8]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:

latest_path = "/content/drive/MyDrive/A_Chess_Transformer/chess_checkpoints/chess_v1_vocab_size=2008_pad_id=2006_d_model=1_024_d_ff=4_096_num_layers=8_latest.pt"

state_dict = torch.load(latest_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(state_dict['model_state'])
model.to('cuda')

AutoregressiveTransformer(
  (token_emb): Embedding(2008, 1024, padding_idx=2006)
  (pos_emb): Embedding(257, 1024)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=4096, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=1024, out_features=2008, bias=False)
)

In [37]:
from GRPO.data import RLThinkDataset

d = RLThinkDataset(GAMES)

In [38]:
!   wget https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar && \
    tar -xf stockfish-ubuntu-x86-64-sse41-popcnt.tar stockfish/stockfish-ubuntu-x86-64-sse41-popcnt


--2025-05-28 22:42:28--  https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/official-stockfish/Stockfish/releases/download/sf_17.1/stockfish-ubuntu-x86-64-sse41-popcnt.tar [following]
--2025-05-28 22:42:28--  https://github.com/official-stockfish/Stockfish/releases/download/sf_17.1/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/20976138/1f1e24b8-d3b0-49bf-92fd-ebd01964592a?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250528%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250528T224228Z&X-Amz-Expires=300&X-Amz-Signature=0c6d4d

In [39]:
path = "stockfish/stockfish-ubuntu-x86-64-sse41-popcnt"

In [40]:
test_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True)

test_batch = next(iter(test_loader))
test_batch.shape

torch.Size([8, 128])

In [41]:
from GRPO.grpo_trainer import GRPOArgs, GRPOTrainer
ref_model = AutoregressiveTransformer(ChessConfig()).to(device)
trainer = GRPOTrainer(model, ref_model, GRPOArgs())

In [42]:
outputs = trainer._generate_completions_and_score(model, test_batch.to(device), path)
loss = trainer._compute_loss(model, outputs)

In [43]:
# Looks correct, simulated environment responses (i.e opponent, in this case the model, playing back) are not given 0 advantage
outputs["advantages"]

tensor([[-3.2200e+00, -0.0000e+00, -3.5800e+00, -0.0000e+00, -2.6350e+00,
         -0.0000e+00, -2.3875e+00, -0.0000e+00, -2.7075e+00, -0.0000e+00,
         -1.6625e+00, -0.0000e+00, -3.1025e+00, -0.0000e+00, -1.7125e+00,
         -0.0000e+00, -1.2625e+00, -0.0000e+00, -8.3750e-01,  0.0000e+00],
        [-1.1400e+00, -0.0000e+00, -5.6000e-01, -0.0000e+00, -1.3450e+00,
         -0.0000e+00, -3.4875e+00, -0.0000e+00, -2.9575e+00, -0.0000e+00,
         -1.9125e+00,  0.0000e+00,  7.7500e-02,  0.0000e+00,  4.6750e-01,
         -0.0000e+00, -1.3125e+00, -0.0000e+00, -8.8750e-01,  0.0000e+00],
        [ 1.0900e+00,  0.0000e+00,  1.7600e+00,  0.0000e+00,  2.3350e+00,
          0.0000e+00,  4.1925e+00,  0.0000e+00,  3.2225e+00,  0.0000e+00,
          1.8075e+00,  0.0000e+00,  3.6750e-01,  0.0000e+00,  7.6750e-01,
          0.0000e+00,  9.0750e-01,  0.0000e+00,  1.3325e+00,  0.0000e+00],
        [ 3.2700e+00,  0.0000e+00,  2.3800e+00,  0.0000e+00,  1.6450e+00,
          0.0000e+00,  1.6825e+00, 

In [55]:
from GRPO.grpo_trainer import GRPOArgs, GRPOTrainer
ref_model = AutoregressiveTransformer(ChessConfig()).to(device)
trainer = GRPOTrainer(model, ref_model, GRPOArgs())

In [56]:

# train_loader = torch.utils.data.DataLoader(d, batch_size=8, shuffle=True)
# trainer.train(train_loader, path)

step      5 | loss   0.3296
step     10 | loss   0.3731
step     15 | loss   1.0404
step     20 | loss   0.2992
step     25 | loss   0.0298
step     30 | loss   3.2278
step     35 | loss 686.2613
step     40 | loss   0.1402
step     45 | loss   1.0516
step     50 | loss   0.2664
step     55 | loss   0.0945
step     60 | loss   8.8808
step     65 | loss  23.4691
step     70 | loss   0.2337
step     75 | loss   2.2716
step     80 | loss   0.2121
step     85 | loss   0.1132
step     90 | loss  22.0905
step     95 | loss   0.2294
step    100 | loss   1.2149
step    105 | loss   0.2266
step    110 | loss   0.2697
step    115 | loss   0.1143
step    120 | loss   0.1210
step    125 | loss   1.0479
step    130 | loss   0.5582
step    135 | loss   0.2016
step    140 | loss   0.1006
step    145 | loss 399.3665
step    150 | loss   0.1906
step    155 | loss   0.5203
step    160 | loss   0.1067
step    165 | loss   2.3041
step    170 | loss   0.2244
step    175 | loss   0.2155
step    180 | loss  

KeyboardInterrupt: 

In [57]:
%cd ..
torch.save({
        'model_state': model.state_dict(),
        'opt_state':  trainer.optimizer.state_dict(),
        'metrics': trainer._metrics,
        'global_step': trainer.global_step,
    }, f"/content/drive/MyDrive/A_Chess_Transformer/grpo_{trainer.global_step}.pt")

/content


In [54]:
%cd /content/RL-Chess-Transformers/


/content/RL-Chess-Transformers


In [ ]:
trainer._metrics["train"]["reward"]

In [33]:
path = "/content/drive/MyDrive/A_Chess_Transformer/grpo_960.pt"
state_dict = torch.load(path, map_location='cuda' if torch.cuda.is_available() else 'cpu')


In [36]:
model_grpo = AutoregressiveTransformer(ChessConfig())
model_grpo.load_state_dict(state_dict['model_state'])
model_grpo.to('cuda')


AutoregressiveTransformer(
  (token_emb): Embedding(2008, 1024, padding_idx=2006)
  (pos_emb): Embedding(257, 1024)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=4096, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (lm_head): Linear(in_features=1024, out_features=2008, bias=False)
)

### Evaluations

In [15]:
from GRPO.evaluations.utils import evaluate_model_vs_engine

In [26]:
!wget https://github.com/fairy-stockfish/Fairy-Stockfish/releases/latest/download/fairy-stockfish-largeboard_x86-64
!chmod +x fairy-stockfish-largeboard_x86-64   # make it executable

--2025-05-29 02:54:58--  https://github.com/fairy-stockfish/Fairy-Stockfish/releases/latest/download/fairy-stockfish-largeboard_x86-64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/fairy-stockfish/Fairy-Stockfish/releases/download/fairy_sf_14/fairy-stockfish-largeboard_x86-64 [following]
--2025-05-29 02:54:58--  https://github.com/fairy-stockfish/Fairy-Stockfish/releases/download/fairy_sf_14/fairy-stockfish-largeboard_x86-64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/138341429/27e76ee3-473a-4aac-b10a-8d99bf817d2b?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250529%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250529T025458Z&X-Amz-Expires=300&X-Amz-Signature=946fc

In [37]:
path_to_fairy = "/content/RL-Chess-Transformers/fairy-stockfish-largeboard_x86-64"
results = evaluate_model_vs_engine(model_grpo)

In [38]:
results

{1: {'win': 7, 'lose': 6, 'draw': 89},
 2: {'win': 4, 'lose': 16, 'draw': 82},
 3: {'win': 0, 'lose': 95, 'draw': 7},
 4: {'win': 0, 'lose': 100, 'draw': 2}}